# Reconnaissance d'entités nommées avec SpaCy

La documentation est accessible ici: https://spacy.io/api

## Imports

In [22]:
from collections import defaultdict
import spacy
from spacy.lang.fr.examples import sentences
!pip install  fr_core_news_md

In [8]:
nlp = spacy.load('fr_core_news_md')

## Exemple sur un corpus de test fourni par SpaCy

In [9]:
# Imprimer le corpus de Spacy
sentences

['Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars',
 "Les voitures autonomes déplacent la responsabilité de l'assurance vers les constructeurs",
 "San Francisco envisage d'interdire les robots coursiers sur les trottoirs",
 'Londres est une grande ville du Royaume-Uni',
 'L’Italie choisit ArcelorMittal pour reprendre la plus grande aciérie d’Europe',
 "Apple lance HomePod parce qu'il se sent menacé par l'Echo d'Amazon",
 "La France ne devrait pas manquer d'électricité cet été, même en cas de canicule",
 'Nouvelles attaques de Trump contre le maire de Londres',
 'Où es-tu ?',
 'Qui est le président de la France ?',
 'Où est la capitale des États-Unis ?',
 'Quand est né Barack Obama ?']

In [10]:
# Isoler la première phrase
sent = sentences[0]
sent

'Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars'

In [11]:
# Traiter la phrase avec Spacy
doc = nlp(sent)

In [12]:
type(doc)

spacy.tokens.doc.Doc

In [13]:
doc.text

'Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars'

In [23]:
doc.to_json()

{'text': 'Quand est né Barack Obama ?',
 'ents': [{'start': 13, 'end': 25, 'label': 'PER'}],
 'sents': [{'start': 0, 'end': 27}],
 'tokens': [{'id': 0,
   'start': 0,
   'end': 5,
   'tag': 'SCONJ',
   'pos': 'SCONJ',
   'morph': '',
   'lemma': 'quand',
   'dep': 'dep',
   'head': 2},
  {'id': 1,
   'start': 6,
   'end': 9,
   'tag': 'AUX',
   'pos': 'AUX',
   'morph': 'Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin',
   'lemma': 'être',
   'dep': 'aux:tense',
   'head': 2},
  {'id': 2,
   'start': 10,
   'end': 12,
   'tag': 'VERB',
   'pos': 'VERB',
   'morph': 'Gender=Masc|Number=Sing|Tense=Past|VerbForm=Part',
   'lemma': 'naître',
   'dep': 'ROOT',
   'head': 2},
  {'id': 3,
   'start': 13,
   'end': 19,
   'tag': 'PROPN',
   'pos': 'PROPN',
   'morph': 'Gender=Fem|Number=Sing',
   'lemma': 'Barack',
   'dep': 'obl:mod',
   'head': 2},
  {'id': 4,
   'start': 20,
   'end': 25,
   'tag': 'ADJ',
   'pos': 'ADJ',
   'morph': 'Gender=Masc|Number=Sing',
   'lemma': 'obama',
   

In [15]:
# Appliquer le test sur toutes les phrases
for sent in sentences:
    doc = nlp(sent)
    entities = []
    for ent in doc.ents:
        entities.append(f"{ent.text} ({ent.label_})")
    if entities:
        print(f"'{doc.text}' contient les entités suivantes : {', '.join(entities)}")
    else:
        print(f"'{doc.text}' ne contient aucune entité")

'Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars' contient les entités suivantes : Apple (ORG)
'Les voitures autonomes déplacent la responsabilité de l'assurance vers les constructeurs' ne contient aucune entité
'San Francisco envisage d'interdire les robots coursiers sur les trottoirs' contient les entités suivantes : San Francisco (LOC)
'Londres est une grande ville du Royaume-Uni' contient les entités suivantes : Londres (LOC), Royaume-Uni (LOC)
'L’Italie choisit ArcelorMittal pour reprendre la plus grande aciérie d’Europe' contient les entités suivantes : L’Italie (LOC), ArcelorMittal (ORG), Europe (LOC)
'Apple lance HomePod parce qu'il se sent menacé par l'Echo d'Amazon' contient les entités suivantes : Apple (ORG), HomePod (MISC), Echo (ORG), Amazon (ORG)
'La France ne devrait pas manquer d'électricité cet été, même en cas de canicule' contient les entités suivantes : La France (LOC)
'Nouvelles attaques de Trump contre le maire de Londres' contient les en

## Appliquer la reconnaissance d'entités nommées sur notre corpus

In [ ]:
# Charger le texte
n=1000000
text = open("../data/all.txt", encoding='utf-8').read()[:n]

FileNotFoundError: [Errno 2] No such file or directory: '../../data/all.txt'

In [25]:
%%time
# Traiter le texte

doc = nlp(text)

CPU times: user 8 μs, sys: 1e+03 ns, total: 9 μs
Wall time: 11.2 μs


NameError: name 'text' is not defined

In [ ]:
# Compter les entités
people = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "PER" and len(ent.text) > 3:
        people[ent.text] += 1

In [ ]:
# Trier et imprimer

sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)

for person, freq in sorted_people[:50]:
    print(f"{person} apparait {freq} fois dans le corpus")

Exercice: essayez de lister les lieux (LOC) et les organisations (ORG) les plus mentionnées dans le corpus